Election de 2022 premier tour

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import os
os.listdir()

['migrations',
 'election2017.ipynb',
 '.DS_Store',
 'alembic.ini',
 'economie',
 'enums',
 'utils',
 'models',
 'election2012.ipynb',
 'exports',
 'labo1.ipynb',
 '.gitignore',
 'population.ipynb',
 'scripts',
 'db',
 'securite.ipynb',
 'chomages',
 'labo2.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'data',
 'chomage.ipynb']

#### Importer la database

In [2]:
from db import SessionLocal
session = SessionLocal()

Import des datasets.

In [3]:
tour = 2
df = pd.read_excel(f'./data/elections/presidentielles-2012-{tour}.xlsx')

Visualise toutes les colones

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

In [ ]:
#df.drop(index=107, inplace=True)

supprimer la colonne Etat saisie d’un DataFrame pandas.

In [ ]:
df = df.drop(columns=["Etat saisie"], errors="ignore")

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

In [ ]:
df.shape

Recuperer les information généraux sur les élections départemental

In [ ]:
df_infos_general = df.iloc[:, :13]
df_infos_general.head()

Renommer les colonnes des information généraux sur les élections départemental

In [ ]:
df_infos_general.columns = [
    "code_dept", "nom_dept", "nb_inscrits", "nb_abstentions",
    "pct_abstentions", "nb_votants", "pct_votants", "nb_blancs_nuls",
    "pct_blancs_nuls_inscrits", "pct_blancs_nuls_votants", "nb_exprimes",
    "pct_exprimes_inscrits", "pct_exprimes_votants"
]
df_infos_general.head()

In [ ]:
df_infos_general.info()

Suppression des données redondantes, risque de multicolinéarité en ML

In [ ]:
colonnes_base = [
    "code_dept", "nom_dept",
    "nb_inscrits", "nb_abstentions", "nb_votants", 
    "nb_blancs_nuls",
]

df_infos_general_base = df_infos_general[colonnes_base]
df_infos_general_base.head()

In [ ]:
df_infos_general_base.shape

In [ ]:
df_departement = df_infos_general_base.iloc[:, :2]

pd.set_option('display.max_rows', None)      # Affiche toutes les lignes
pd.set_option('display.max_columns', None)   # Affiche toutes les colonnes
pd.set_option('display.width', None)         # Évite le retour à la ligne automatique
pd.set_option('display.max_colwidth', None)  # Affiche tout le contenu des cellules

print(df_departement)
#df_departement.head()

In [ ]:
df_stat_elections = df_infos_general_base.drop(columns=["nom_dept"])
df_stat_elections.head()

Tratification pour consever les proportions

In [ ]:
import pandas as pd

# Colonnes globales que l'on veut garder
cols_globales = ['Code du département', 'Libellé du département']

# Colonnes répétitives candidats
df_candidates = df.iloc[:, 13:]  # après % Exp/Vot
cols_per_candidate = 6
num_candidates = df_candidates.shape[1] // cols_per_candidate

dfs_list = []

for i in range(num_candidates):
    start_col = i * cols_per_candidate
    end_col = start_col + cols_per_candidate
    df_cand = df_candidates.iloc[:, start_col:end_col].copy()
    df_cand.columns = ['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    # Ajouter les informations de département
    df_cand = pd.concat([df[cols_globales], df_cand], axis=1)
    
    dfs_list.append(df_cand)

# Combiner tous les candidats
df_simple = pd.concat(dfs_list, ignore_index=True)

# Colonnes à garder
cols_to_keep = ['Code du département', 'Libellé du département', 'Sexe', 'Nom', 'Prenom', 'Voix']

# Nouveau dataframe simplifié
df_candidat_resultat = df_simple[cols_to_keep]

# Afficher toutes les lignes
pd.set_option('display.max_rows', 10)  # ou un nombre supérieur à 100
pd.set_option('display.max_columns', None)  # pour voir toutes les colonnes
pd.set_option('display.width', None)        # pour éviter la coupure à la largeur de l'écran



#df_candidat_resultat = df_candidat_resultat.drop(index=215)
# Affichage des 5 premières lignes
df_candidat_resultat.head(1000)


In [ ]:
from datetime import date

from utils.election_importer import ElectionImporter
from enums.type_election import TypeElection

# =====================================================
# Instanciation
# =====================================================
importer = ElectionImporter()

# =====================================================
# Élection
# =====================================================
if tour == 1:
    election = importer.get_or_create_election(
        election_date=date(2012, 4, 22),  # 1er tour présidentielle 2012
        type_election=TypeElection.PRESIDENTIELLE,
        tour=tour
    )

elif tour == 2:
    election = importer.get_or_create_election(
        election_date=date(2012, 5, 6),  # 2nd tour présidentielle 2012
        type_election=TypeElection.PRESIDENTIELLE,
        tour=tour
    )

# =====================================================
# Départements
# =====================================================
#df_departement = edf.df_departement
if df_departement is not None and not df_departement.empty:
    importer.import_departements(df_departement)

# =====================================================
# Statistiques par département
# =====================================================
#df_stat_elections = edf.df_stat_elections
if df_stat_elections is not None and not df_stat_elections.empty:
    importer.import_stats(df_stat_elections, election.id)

# =====================================================
# Candidats & résultats
# =====================================================
#df_candidat_resultat = edf.df_candidat_resultat
if df_candidat_resultat is not None and not df_candidat_resultat.empty:
    importer.import_candidats_resultats(df_candidat_resultat, election.id)

print("✅ Import élection 2017 – Tour 1 terminé avec succès")
